# 영상의 기하학적 변환

### 회전 변환

In [1]:
import cv2

In [2]:
src = cv2.imread('./data/tekapo.bmp')

h, w ,c = src.shape
cp = h/2, w/2

# getRotationMatrix2D(center좌표, angle, scale)
M = cv2.getRotationMatrix2D(cp, 20, 1)

dst = cv2.warpAffine(src, M, (0,0))

cv2.imshow('src',src)
cv2.imshow('dst',dst)

cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
dst1 = cv2.rotate(src, cv2.ROTATE_90_CLOCKWISE)
dst2 = cv2.rotate(src, cv2.ROTATE_180)
dst3 = cv2.rotate(src, cv2.ROTATE_90_COUNTERCLOCKWISE)

cv2.imshow('src',src)
cv2.imshow('dst1',dst1)
cv2.imshow('dst2',dst2)
cv2.imshow('dst3',dst3)

cv2.waitKey()
cv2.destroyAllWindows()

### 대칭 변환

In [4]:
src = cv2.imread('./data/eastsea.bmp')

cv2.imshow('src',src)

for flip_code in [1,0,-1]: # 그대로, 상하대칭, 상하좌우대칭
    dst = cv2.flip(src, flip_code)
    
    desc = "flip code : {}".format( flip_code ) 
    cv2.putText(dst, desc, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,0,0),1,cv2.LINE_AA)
    cv2.imshow('dst',dst)
    cv2.waitKey()
    
cv2.destroyAllWindows()

### 투시 변환

In [5]:
import numpy as np

src = cv2.imread('./data/card.bmp')

cnt = 0
src_pts = np.zeros([4,2], dtype=np.float32)

def on_mouse(event, x, y, flags, params):
    
    global src_pts, cnt
    
    if event == cv2.EVENT_LBUTTONDOWN :
        
        if(cnt < 4):
            src_pts[cnt] = np.array([x,y],dtype=np.float32)
            cnt=cnt+1;
            
            cv2.circle(src,(x,y),5 ,(255,255,0),-1)
            cv2.imshow('src',src)
        
        if(cnt == 4):
            w, h = (200, 300)
            dst_pts = np.array([[0,0],[w-1,0],[w-1,h-1],[0,h-1]], dtype=np.float32)
            M = cv2.getPerspectiveTransform(src_pts, dst_pts)
            dst = cv2.warpPerspective(src, M, (w,h))
            cv2.imshow('dst',dst)
            

cv2.imshow("src",src)
cv2.setMouseCallback("src",on_mouse)

cv2.waitKey()
cv2.destroyAllWindows()

# 에지 검출과 응용

In [6]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('image load failed')
    sys.exit()

m = np.array([[-1, 0, 1],
      [-2, 0, 2],
      [-1, 0, 1]], dtype=np.float32)

dx, dy = cv2.filter2D(src, -1, m, delta=128), cv2.filter2D(src, -1, m.T, delta=128)
cv2.imshow('src', src)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)

cv2.waitKey()
cv2.destroyAllWindows()

### 마스크 기반 에지 검출 - 소벨 마스크

In [7]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('image load failed')
    sys.exit()

dx = cv2.Sobel(src, -1, 1, 0, delta=128)
dy = cv2.Sobel(src, -1, 0, 1, delta=128)

cv2.imshow('src', src)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)

cv2.waitKey()
cv2.destroyAllWindows()  

In [8]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('image load failed')
    sys.exit()

dx = cv2.Sobel(src, cv2.CV_32FC1, 1, 0)
dy = cv2.Sobel(src, cv2.CV_32FC1, 0, 1)

mag = cv2.magnitude(dx, dy)
mag = np.clip(mag, 0, 255).astype(np.uint8)

_, edge = cv2.threshold(mag, 90, 255, cv2.THRESH_BINARY)

cv2.imshow('src', src)
cv2.imshow('mag', mag)
cv2.imshow('edge', edge)

cv2.waitKey()
cv2.destroyAllWindows()  

### 캐니 에즈 검출기

In [9]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('image load failed')
    sys.exit()

dst1 = cv2.Canny(src, 50, 100)
dst2 = cv2.Canny(src, 60, 180)

cv2.imshow('src', src)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)

cv2.waitKey()
cv2.destroyAllWindows()  

### 허프 변환 직선 검출

In [10]:
import math
src = cv2.imread('./data/building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('image load failed')
    sys.exit()

edge = cv2.Canny(src, 50, 150)
rho = 1
theta = math.pi/180

lines = cv2.HoughLines(edge, rho, theta, 250)

dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

if lines is not None:
    for i in range(lines.shape[0]):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        cos_t = math.cos(theta)
        sin_t = math.sin(theta)
        
        x0, y0 = rho * cos_t, rho * sin_t
        alpha = 1000
        
        pt1 = (int(x0 - alpha * sin_t), int(y0 + alpha * cos_t))
        pt2 = (int(x0 + alpha * sin_t), int(y0 - alpha * cos_t))
        
        cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('edge', edge)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()  

In [11]:
src = cv2.imread('./data/building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('image load failed')
    sys.exit()

edge = cv2.Canny(src, 50, 150)
rho = 1
theta = math.pi/180

lines = cv2.HoughLinesP(edge, rho, theta, 220, minLineLength=50,maxLineGap=5)

dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR)

if lines is not None:
    for i in range(lines.shape[0]):
        pt1 = (lines[i][0][0], lines[i][0][1])
        pt2 = (lines[i][0][2], lines[i][0][3])
        cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()  

### 허프 변환 원 검출

In [12]:
src = cv2.imread('./data/coins.png', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('image load failed')
    sys.exit()

blurred = cv2.blur(src, (3, 3))
circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=150, param2=30)

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

if circles is not None:
    for cx, cy, radius in circles[0]:
        cv2.circle(dst, (np.uint32(cx), np.uint32(cy)), np.uint32(radius), (0,0,255), 2)

cv2.imshow('src', src)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()  